In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

# Function Calling 介绍

**注意**：在 OpenAI 中并不是所有模型都支持`Function Calling`，而默认的名称会随着版本迭代发生不可预测的变化，因此最好指定具体模型名称。

<div class="alert alert-success">

**以下模型支持 `Funcition Calling`**：<br>
gpt-4, gpt-4-turbo-preview, gpt-4-0125-preview, gpt-4-1106-preview, gpt-4-0613, gpt-3.5-turbo, gpt-3.5-turbo-0125, gpt-3.5-turbo-1106, and gpt-3.5-turbo-0613

**以下模型支持`并行调用`**：<br>
gpt-4-turbo-preview, gpt-4-0125-preview, gpt-4-1106-preview, gpt-3.5-turbo-0125, and gpt-3.5-turbo-1106

</div>

# OpenAI 函数回调

## 定义工具

<div class="alert alert-warning">
<b>注意：</b><br>
    
1. OpenAI使用的Tools名称类似于一个函数命名，必须使用下划线或ASCII码，**不能使用中文**！<br>
但描述部份可以使用中文。
2. 定义工具时，请**至少包含一个参数**（即使你不使用这个参数），否则调用 GPT 时可能会抛出异常
</div>

In [2]:
import json
import random

from langchain.tools import tool
from langchain_core.utils.function_calling import convert_to_openai_tool

# @tool(args_schema=WhereIsCatSchema)
@tool("WhereIsCatHidding")
def where_is_cat_hiding(idea: str) -> str:
    """从这些地方选择猫躲藏的地方"""
    return random.choice(["在床底下吗？", "在书架中吗？", "在阳台吗？"])

@tool("GetItem")
def get_items(place: str) -> str:
    """使用这个工具去仔细查看猫躲藏的地方"""
    if "床底" in place:  # For under the bed
        return "发现几只臭袜子和鞋，但没发现猫"
    elif "书架" in place:  # For 'shelf'
        return "发现一些漫画书、图册和铅笔，但没发现猫"
    elif "阳台" in place: 
        return "我找到猫了"
    else:  # 如果智能体决定找其他地方
        return "找到屋外了，但没发现猫"

## convert_to_openai_tool

In [3]:
print(json.dumps(convert_to_openai_tool(where_is_cat_hiding), indent=2, ensure_ascii=False))

{
  "type": "function",
  "function": {
    "name": "WhereIsCatHidding",
    "description": "WhereIsCatHidding(idea: str) -> str - 从这些地方选择猫躲藏的地方",
    "parameters": {
      "type": "object",
      "properties": {
        "idea": {
          "type": "string"
        }
      },
      "required": [
        "idea"
      ]
    }
  }
}


## 调用 OpenAI

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

In [5]:
resp = llm.invoke(
    "猫藏在哪里？",
    tools=[
        convert_to_openai_tool(where_is_cat_hiding),
        convert_to_openai_tool(get_items)
    ])
print(resp)

content='' additional_kwargs={'tool_calls': [{'id': 'call_97C4J3CcEwR3ryJOWbpKf0Ep', 'function': {'arguments': '{"idea":"找猫"}', 'name': 'WhereIsCatHidding'}, 'type': 'function'}]}


以下方法是等价的：
```python
# bind
llm.bind(tools=[convert_to_openai_tool(where_cat_is_hiding)]).invoke("猫藏在哪里？")
# bind_tools
llm.bind_tools([convert_to_openai_tool(where_cat_is_hiding)]).invoke("猫藏在哪里？")
```

# 解析 OpenAI 函数和参数

## 解析为 JSON

In [6]:
from langchain.output_parsers.openai_tools import JsonOutputToolsParser

JsonOutputToolsParser().invoke(resp)

[{'type': 'WhereIsCatHidding', 'args': {'idea': '找猫'}}]

## 提取 JSON 特定键

In [7]:
from langchain.output_parsers.openai_tools import JsonOutputKeyToolsParser

JsonOutputKeyToolsParser(key_name="WhereIsCatHidding").invoke(resp)

[{'idea': '找猫'}]

## 提取 Pydantic 对象

In [8]:
from langchain_core.pydantic_v1 import BaseModel, Field

class WhereIsCatHidding(BaseModel):
    """找猫"""

    idea: str = Field(..., description="寻找猫的线索")
    
class GetItems(BaseModel):
    """看看有什么发现"""

    place: str = Field(..., description="地点的描述")

In [9]:
from langchain.output_parsers.openai_tools import PydanticToolsParser

parser = PydanticToolsParser(tools=[WhereIsCatHidding, GetItems])

In [10]:
tools = [convert_to_openai_tool(t) for t in [where_is_cat_hiding, get_items]]
chain = llm.bind(seed=42, tools=tools) | parser

In [11]:
resp = chain.invoke("猫藏在哪里？")
print(resp)

[WhereIsCatHidding(idea='找猫')]


## 运行函数

In [504]:
for obj in resp:
    if(isinstance(obj, WhereIsCatHidding)):
        print(where_cat_is_hiding(obj.idea))
    elif(isinstance(obj, GetItems)):
        print(get_item(obj.place))
    else:
        print("没有工具需要执行")

在床底下


## 小结

如前文所示：GPT模型会在每次检测到需要调用函数时返回这个函数的名称和参数，但需要你自己在代码中去运行工具函数。
通常，你还需要将函数执行结果反馈给GPT，然后生成新的结果。

智能体将自动化上述过程。

# 使用 OpenAI Tool 自定义智能体

## Prompt

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你正在与我玩一个找猫的游戏",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

## Tool

In [13]:
tools=[
        convert_to_openai_tool(where_is_cat_hiding),
        convert_to_openai_tool(get_items)
    ]

## LLM

In [507]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125").bind_tools(tools)

In [508]:
llm.invoke("猫呢?")

AIMessage(content='猫可能躲在哪里呢？你有什么想法？')

In [509]:
chain = {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    } | prompt | llm

chain.invoke({"input": "猫在哪?", "intermediate_steps": ""})

AIMessage(content='让我来帮你找找看，让我想想... 嗯，我有一个主意！让我们从这些地方选择猫躲藏的地方吧。', additional_kwargs={'tool_calls': [{'id': 'call_DNdQbXoBNNiOGdJOhU6L1mUe', 'function': {'arguments': '{"idea":"under the table"}', 'name': 'WhereIsCatHidding'}, 'type': 'function'}]})

## Agent

In [510]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm
    | OpenAIToolsAgentOutputParser()
)

## AgentExecutor

**注意** 在这个例子中有两处对工具的声明：

- 第一次是作为提示语的一部份提供给 OpenAI；
- 第二次是作为可执行的 Python 函数清单提供给 AgentExecutor（此时并不需要转换为 OpenAI 格式）。

In [511]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=[where_is_cat_hiding, get_items], verbose=True)

## 运行

In [512]:
agent_executor.invoke({"input": "猫在哪里?"})



> Entering new AgentExecutor chain...

Invoking: `WhereIsCatHidding` with `{'idea': '在书架上'}`


阳台
Invoking: `GetItem` with `{'place': '书架'}`


发现一些漫画书、图册和铅笔，但没发现猫
Invoking: `GetItem` with `{'place': '阳台'}`


我找到猫了猫可能在阳台上，我去找找看。

> Finished chain.


{'input': '猫在哪里?', 'output': '猫可能在阳台上，我去找找看。'}

# OpenAI 回调工具的推理

In [41]:
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_core.utils.function_calling import convert_to_openai_tool

@tool("data_incoming")
def get_data_incoming(idea: str) -> str:
    """仅查询XX月份销售收入"""
    pass

@tool("data_default")
def default_tool(place: str) -> str:
    """如果无法用已有知识推理解决，就请使用默认工具"""
    pass

llm = ChatOpenAI(model_name="gpt-4-0125-preview")

## 刚好匹配正确工具时

In [42]:
llm.invoke(
    "8月份销售总额是多少？",
    tools=[
        convert_to_openai_tool(get_data_incoming),
    ])

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_49u1uOvkKz2MU2b4RECDr4gK', 'function': {'arguments': '{"idea":"8月份"}', 'name': 'data_incoming'}, 'type': 'function'}]})

## 不需要工具时

In [35]:
llm.invoke(
    "给我讲个关于程序员的笑话",
    tools=[
        convert_to_openai_tool(get_data_incoming),
    ])

AIMessage(content='有这么一个笑话：\n\n程序员的妻子让他去超市买一瓶牛奶，然后又加了一句：“如果有鸡蛋，买六个。”\n\n程序员回来了，手里拿着六瓶牛奶。\n\n妻子问：“为什么买这么多牛奶？”\n\n程序员回答：“因为他们有鸡蛋。”')

<div class="alert alert-warning">
<b>⚠️ 注意</b>
gpt-3.5-turbo 会忘记自己会讲笑话，然后错误选择工具来解决问题。
</div>

## 没有趁手工具可用时

In [36]:
llm.invoke(
    "我们现在有哪些产品？",
    tools=[
        convert_to_openai_tool(get_data_incoming),
    ])

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_D6NbxnCsF2Wy73MVSvtDujeK', 'function': {'arguments': '{"idea":"products"}', 'name': 'data_incoming'}, 'type': 'function'}]})

<div class="alert alert-warning">
<b>⚠️ 注意</b>
即使 gpt-4-0125-preview 仍然无法会尝试选择工具解决问题，大概意识到单凭自己的知识无法回答这个问题。
</div>

## 路由到默认工具

In [39]:
llm.invoke(
    "8月份销售总额是多少？",
    tools=[
        convert_to_openai_tool(get_data_incoming),
        convert_to_openai_tool(get_data_default)
    ])

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_hm1ZRfVp32hWn9qAp2Q8Ip4N', 'function': {'arguments': '{"idea":"8月份销售总额"}', 'name': 'data_incoming'}, 'type': 'function'}]})

In [43]:
llm.invoke(
    "我们销售最好的产品是哪个？",
    tools=[
        convert_to_openai_tool(get_data_incoming),
        convert_to_openai_tool(get_data_default)
    ])

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_fPUidO1OismNbboXWhhZ6P1u', 'function': {'arguments': '{"place":"销售最好的产品"}', 'name': 'data_default'}, 'type': 'function'}]})

<div class="alert alert-success">
    <b>🏅️ 正确选择到默认工具</b><br>
    GPT4 提供了一种可能性：通过默认工具路由到更复杂的智能体来解决问题！
</div>

# 创建智能体的帮助函数

## create_openai_tools_agent

### 准备

In [545]:
from langchain import hub
from langchain.agents import AgentExecutor, Tool, create_openai_tools_agent
from langchain.prompts import ChatPromptTemplate
from langchain.tools import tool, StructuredTool
from langchain_core.callbacks import Callbacks
from langchain_openai import ChatOpenAI

### Prompt

从 `langsmith` 的 hub 中直接提取提示语：

In [546]:
prompt = hub.pull("hwchase17/openai-tools-agent")
print(prompt.messages) # to see the prompt

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(variable_name='agent_scratchpad')]


### LLM

**注意** 此处不需要将工具使用说明提供给 OpenAI，而是交给后面的 `create_openai_tols_agent` 来做这件事：

In [565]:
model = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0, streaming=True)

### Agent

In [566]:
tools = [where_is_cat_hiding, get_items]

In [567]:
# tools：提供工具说明
agent = create_openai_tools_agent(model, tools, prompt)

### AgentExecutor

In [568]:
# tools：提供调用清单
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

### invoke

In [570]:
agent_executor.invoke({"input": "猫呢?"})



> Entering new AgentExecutor chain...

Invoking: `WhereIsCatHidding` with `{'idea': '在书架上'}`


在床底下吗？
Invoking: `GetItem` with `{'place': '床底下'}`


发现几只臭袜子和鞋，但没发现猫
Invoking: `GetItem` with `{'place': '书架上'}`
responded: 猫并不在床底下。我将继续帮您寻找猫。

发现一些漫画书、图册和铅笔，但没发现猫
Invoking: `GetItem` with `{'place': '沙发下'}`
responded: 猫并不在床底下。我将继续帮您寻找猫。

找到屋外了，但没发现猫
Invoking: `WhereIsCatHidding` with `{'idea': '在厨房柜子里'}`
responded: 在书架上发现了一些漫画书、图册和铅笔，但没有发现猫。在沙发下也找到了，但没有发现猫。让我再想想其他地方。

在床底下吗？
Invoking: `GetItem` with `{'place': '厨房柜子里'}`


找到屋外了，但没发现猫
Invoking: `GetItem` with `{'place': '床底下'}`
responded: 猫并不在厨房柜子里。我将继续帮您寻找猫。

发现几只臭袜子和鞋，但没发现猫
Invoking: `GetItem` with `{'place': '客厅窗台'}`
responded: 猫并不在厨房柜子里。我将继续帮您寻找猫。

找到屋外了，但没发现猫
Invoking: `WhereIsCatHidding` with `{'idea': '在衣柜里'}`
responded: 在床底下发现了一些猫毛和鞋子，但没有发现猫。在客厅窗台也找到了，但没有发现猫。让我再想想其他地方。

在阳台吗？
Invoking: `GetItem` with `{'place': '衣柜里'}`


找到屋外了，但没发现猫
Invoking: `GetItem` with `{'place': '阳台'}`
responded: 猫并不在衣柜里。我将继续帮您寻找猫。

我找到猫了
Invoking: `GetItem` w

{'input': '猫呢?', 'output': '成功找到猫了！它现在已经找到了。有什么我可以帮您的吗？'}

### astream

打印智能体的日志时经常涉及到多层次的 JSON 数据，这里有一些技巧：<br>
例如，首先使用 `pprint` 打印第1层深度的信息，再根据需要逐层深入，对于调试会非常有帮助。

In [571]:
import pprint

chunks = []

async for chunk in agent_executor.astream({"input": "猫呢?"}):
    chunks.append(chunk)
    print("-" * 80)
    pprint.pprint(chunk, depth=1)



> Entering new AgentExecutor chain...
--------------------------------------------------------------------------------
{'actions': [...], 'messages': [...]}

Invoking: `WhereIsCatHidding` with `{'idea': 'under the bed'}`


在阳台吗？--------------------------------------------------------------------------------
{'messages': [...], 'steps': [...]}
--------------------------------------------------------------------------------
{'actions': [...], 'messages': [...]}

Invoking: `GetItem` with `{'place': 'balcony'}`


找到屋外了，但没发现猫--------------------------------------------------------------------------------
{'messages': [...], 'steps': [...]}
--------------------------------------------------------------------------------
{'actions': [...], 'messages': [...]}

Invoking: `WhereIsCatHidding` with `{'idea': 'in the closet'}`
responded: 猫可能不在阳台上。我会继续搜索其他地方。

在阳台吗？--------------------------------------------------------------------------------
{'messages': [...], 'steps': [...]}
-----------------

Stopping agent prematurely due to triggering stop condition


--------------------------------------------------------------------------------
{'actions': [...], 'messages': [...]}

Invoking: `WhereIsCatHidding` with `{'idea': 'in the laundry room'}`
responded: 猫可能不在卧室里。我会继续搜索其他地方。

在书架中吗？--------------------------------------------------------------------------------
{'messages': [...], 'steps': [...]}


> Finished chain.
--------------------------------------------------------------------------------
{'messages': [...], 'output': 'Agent stopped due to max iterations.'}


In [572]:
chunks[0]['actions']

[OpenAIToolAgentAction(tool='WhereIsCatHidding', tool_input={'idea': 'under the bed'}, log="\nInvoking: `WhereIsCatHidding` with `{'idea': 'under the bed'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_H8mO7yAfYgDLXrfkTX27vB1H', 'function': {'arguments': '{"idea":"under the bed"}', 'name': 'WhereIsCatHidding'}, 'type': 'function'}]})], tool_call_id='call_H8mO7yAfYgDLXrfkTX27vB1H')]

In [485]:
async for chunk in agent_executor.astream({"input": "猫呢?"}):
    # Agent Action
    if "actions" in chunk:
        for action in chunk["actions"]:
            print(f"Calling Tool: `{action.tool}` with input `{action.tool_input}`")
    # Observation
    elif "steps" in chunk:
        for step in chunk["steps"]:
            print(f"Tool Result: `{step.observation}`")
    # Final result
    elif "output" in chunk:
        print(f'Final Output: {chunk["output"]}')
    else:
        raise ValueError()
    print("-" * 80)



> Entering new AgentExecutor chain...
Calling Tool: `WhereIsCatHidding` with input `{'idea': '猫呢?'}`
--------------------------------------------------------------------------------

Invoking: `WhereIsCatHidding` with `{'idea': '猫呢?'}`


在书架中Tool Result: `在书架中`
--------------------------------------------------------------------------------
猫躲在书架中。

> Finished chain.
Final Output: 猫躲在书架中。
--------------------------------------------------------------------------------


## create_openai_tools_agent(ChatZhipuAI)

In [ ]:
!poetry add zhipuai

In [ ]:
!poetry add langchain_chinese@latest

In [594]:
from langchain import hub
from langchain.agents import AgentExecutor, Tool, create_openai_tools_agent
from langchain.prompts import ChatPromptTemplate
from langchain.tools import tool, StructuredTool
from langchain_core.callbacks import Callbacks

In [595]:
tools = [where_is_cat_hiding, get_items]

In [596]:
from langchain_chinese import ChatZhipuAI

llm_zhipu = ChatZhipuAI(model="glm-4").bind(tools=tools)

In [597]:
from langchain.agents import AgentExecutor

prompt = hub.pull("hwchase17/openai-tools-agent")
agent = create_openai_tools_agent(llm_zhipu, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=[where_is_cat_hiding, get_items], verbose=True)

In [598]:
agent_executor.invoke({"input": "猫在哪里?"})



> Entering new AgentExecutor chain...

Invoking: `WhereIsCatHidding` with `{'idea': '猫可能在沙发下面、床底下、衣柜里或者书架后面'}`


在床底下吗？
Invoking: `GetItem` with `{'place': '床底下'}`


发现几只臭袜子和鞋，但没发现猫没有，我检查了床底下，只发现了几只臭袜子和鞋，猫不在那里。

> Finished chain.


{'input': '猫在哪里?', 'output': '没有，我检查了床底下，只发现了几只臭袜子和鞋，猫不在那里。'}

## create_react_agent

In [540]:
from langchain.agents import AgentExecutor, Tool, create_react_agent

In [541]:
prompt = hub.pull("hwchase17/react")
tools = [where_is_cat_hiding, get_items]
model = ChatOpenAI(temperature=0, streaming=False).bind(seed=42)

In [542]:
agent = create_react_agent(model, tools, prompt)

In [543]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [544]:
async for chunk in agent_executor.astream({"input": "猫呢?"}):
    # Agent Action
    if "actions" in chunk:
        for action in chunk["actions"]:
            print(f"Calling Tool: `{action.tool}` with input `{action.tool_input}`")
    # Observation
    elif "steps" in chunk:
        for step in chunk["steps"]:
            print(f"Tool Result: `{step.observation}`")
    # Final result
    elif "output" in chunk:
        print(f'Final Output: {chunk["output"]}')
    else:
        raise ValueError()
    print("-" * 80)



> Entering new AgentExecutor chain...
Calling Tool: `WhereIsCatHidding` with input `None
`
--------------------------------------------------------------------------------
I need to find out where the cat is hiding.
Action: WhereIsCatHidding
Action Input: None
在书架中吗？Tool Result: `在书架中吗？`
--------------------------------------------------------------------------------
Calling Tool: `GetItem` with input `书架

`
--------------------------------------------------------------------------------
The cat might be hiding in the bookshelf.
Action: GetItem
Action Input: 书架

发现一些漫画书、图册和铅笔，但没发现猫Tool Result: `发现一些漫画书、图册和铅笔，但没发现猫`
--------------------------------------------------------------------------------
Calling Tool: `WhereIsCatHidding` with input `None

`
--------------------------------------------------------------------------------
The cat is not in the bookshelf. I need to keep looking.
Action: WhereIsCatHidding
Action Input: None

在书架中吗？Tool Result: `在书架中吗？`
----------------------------

## create_react_agent(ChatZhipuAI)

In [540]:
from langchain.agents import AgentExecutor, Tool, create_react_agent

In [599]:
prompt = hub.pull("hwchase17/react")
tools = [where_is_cat_hiding, get_items]
model = ChatZhipuAI(temperature=0.1)

In [600]:
agent = create_react_agent(model, tools, prompt)

In [601]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [602]:
async for chunk in agent_executor.astream({"input": "猫呢?"}):
    # Agent Action
    if "actions" in chunk:
        for action in chunk["actions"]:
            print(f"Calling Tool: `{action.tool}` with input `{action.tool_input}`")
    # Observation
    elif "steps" in chunk:
        for step in chunk["steps"]:
            print(f"Tool Result: `{step.observation}`")
    # Final result
    elif "output" in chunk:
        print(f'Final Output: {chunk["output"]}')
    else:
        raise ValueError()
    print("-" * 80)



> Entering new AgentExecutor chain...
Calling Tool: `WhereIsCatHidding` with input `猫可能躲藏的地方
Observation`
--------------------------------------------------------------------------------
Thought: 首先，我需要确定猫可能躲藏的地方。我可以使用“WhereIsCatHidding”函数来得到一些建议。
Action: WhereIsCatHidding
Action Input: 猫可能躲藏的地方
Observation在书架中吗？Tool Result: `在书架中吗？`
--------------------------------------------------------------------------------
Calling Tool: `GetItem` with input `书架
Observation`
--------------------------------------------------------------------------------
Thought: 如果猫躲在书架上，我可以尝试获取书架上的物品来查看猫的具体位置。我可以使用“GetItem”函数。
Action: GetItem
Action Input: 书架
Observation发现一些漫画书、图册和铅笔，但没发现猫Tool Result: `发现一些漫画书、图册和铅笔，但没发现猫`
--------------------------------------------------------------------------------
Calling Tool: `WhereIsCatHidding` with input `其他可能的猫躲藏地点
Observation`
--------------------------------------------------------------------------------
Thought: Thought: 猫可能不在书架上，我需要考虑其他可能的躲藏地点。
Action: WhereIsC